### Find Original Distance (GAS-PIPELINES)

In [128]:
import pandas as pd
import numpy as np
import math
import json
import glob


import rasterio
import shapefile
from IPython.display import display

In [129]:
def deg2rad(deg):
    return (deg * (math.pi/180))

def get_distance(p, q):
    # lat1, lon1 = p
    # lat2, lon2 = q

    lon1, lat1 = p
    lon2, lat2 = q
    #const [lat1, lon1] = pair1, [lat2, lon2] = pair2;
    #const R = 6371; // Radius of the earth in km.

    R = 6371

    dLat = deg2rad(lat2 - lat1)
    dLon = deg2rad(lon2 - lon1)
    a = math.sin(dLat/2) * math.sin(dLat/2) + math.cos(deg2rad(lat1)) * math.cos(deg2rad(lat2)) * math.sin(dLon/2) * math.sin(dLon/2);
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a));

    return R * c

def find_distance(file_name):
    f = open(file_name)
    data = json.load(f)
 

    points = []
    for feature in data['features']:
        points = feature['geometry']['coordinates']
        # print(len(feature['geometry']['coordinates']))
    
    points = np.array(points)

    distance = 0
    for index in range(1, 2):
        # print(points[index])

        # distance = 0
        # for k in range(len(points)):
        current_path_list = []
        for i in range(1, len(points[index])):
            p = points[index][i - 1]
            q = points[index][i]
        
            # print(p, q)
            current_path_list.append(p)
            
            #distance += math.sqrt(math.pow((p[0] - q[0]), 2) + math.pow((p[1] - q[1]), 2))
            distance += get_distance(p, q)
    return distance, current_path_list

In [130]:
path = 'raster.tif' 

data = rasterio.open(path)
print(data.bounds)

extent = data.bounds

left, bottom, right, top = extent[0], extent[1], extent[2], extent[3]
print(left, bottom, right, top)

BoundingBox(left=282680.0, bottom=2035820.0, right=387680.0, top=2118550.0)
282680.0 2035820.0 387680.0 2118550.0


In [131]:
# reading LULC data
area_cover = 'mask.tif'
area = rasterio.open(area_cover, count = 1)

area = np.array(area.read(1))

mask_data = area
print(mask_data.shape)
print(np.unique(mask_data, return_counts = True))

mask_data = mask_data.T

(8273, 10500)
(array([0, 1, 2, 3, 4, 5, 6, 7], dtype=int16), array([     588,  7839581,  3928816, 17345398, 20102336,  2296551,
        6085701, 29267529]))


In [132]:
index = 0
files = glob.glob('OUTPUTS/main_road.geojson')
print(files)
for f in files:
    dist, current_path_list = find_distance(f)
    # name = files[index].split('.')[0].title()
    print(f'Distance = {dist} Kms')
    # index = index + 1

['OUTPUTS/main_road.geojson']
Distance = 94.45558087050145 Kms


/home/shrayank_mistry/Modules/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [133]:
MAIN_KILO = dist
print(MAIN_KILO)

94.45558087050145


#### Main-Road Analysis

In [134]:
files = glob.glob('OUTPUTS/main_road_prj.geojson')
# print(files)

current_path_list = []

for file_name in files:

    f = open(file_name)
    data = json.load(f)
 
    points = []
    for feature in data['features']:
        points = feature['geometry']['coordinates']
    
    points = np.array(points)

    distance = 0
    for index in range(1, 2):
        current_path_list = []
        for i in range(1, len(points[index])):
            p = points[index][i - 1]
        
            current_path_list.append(p)

/home/shrayank_mistry/Modules/virtualenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  from ipykernel import kernelapp as app


In [135]:
pixels_dict = {}
for i in range(len(current_path_list)):

    ext_i, ext_j = current_path_list[i]
    # print(ext_i, ext_j)

    pi, pj = (ext_i - left) / 10, ((ext_j - top) / 10) * -1
    pixels_dict[ext_i, ext_j] = [pi, pj]


In [136]:
classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

check_pixels = {
    1: 0,
    2: 0,
    3: 0,
    4: 0,
    5: 0,
    6: 0,
    7: 0,
    8: 0,
}


total_pxs = len(current_path_list)
for i in range(len(current_path_list)):
    ext_i, ext_j = current_path_list[i]
    pi, pj = pixels_dict[ext_i, ext_j]

    pi, pj = int(pi), int(pj)
    cs = mask_data[pi][pj]

    check_pixels[cs] += 1
    check_pixels[8] += 1


df_dict_pixels = {}
for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

for key, value in check_pixels.items():
    check_pixels[key] = np.round(((value / total_pxs) * 100), 2)

df_dict_perc = {}
for key, value in check_pixels.items():
    df_dict_perc[classes[key]] = f'{check_pixels[key]}%'

df_dict_dist = {}
for key, value in check_pixels.items():
    df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * road_distance) / 100)} Kms'


list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
display(route_stats)


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,106 pxs,4 pxs,9 pxs,6 pxs,434 pxs,0 pxs,289 pxs,848 pxs
ROAD PERCENTAGE,12.5%,0.47%,1.06%,0.71%,51.18%,0.0%,34.08%,100.0%
ROAD KILOMETERS,11.81 Kms,0.44 Kms,1.0 Kms,0.67 Kms,48.34 Kms,0.0 Kms,32.19 Kms,94.46 Kms


#### Route-Optimization Analysis

In [137]:
namming_dict = {
    1: 'A* Chebyshev distance',
    2: 'A* Diagonal distance',
    3: 'A* Euclidean distance',
    4: 'A* Manhattan distance',
    5: 'A* Proposed',
    6: 'Dijkstra\'s algorithm',
}

paths_dict = {
    1: 'paths/a-star-chesb',
    2: 'paths/a-star-diag',
    3: 'paths/a-star-euc',
    4: 'paths/a-star-man',
    5: 'paths/a-star-new',
    6: 'paths/dijk-route',
}

shape_dict = {
    1: 'routes-shape/a-star-chesb',
    2: 'routes-shape/a-star-diag',
    3: 'routes-shape/a-star-euc',
    4: 'routes-shape/a-star-man',
    5: 'routes-shape/a-star-new',
    6: 'routes-shape/dijk-route',
}

In [138]:
kilometers = []

In [139]:
for p in range(1, 7):
    path = paths_dict[p]
    path_list = []

    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()

    # new_path_list = []
    # for i in range(1, len(path_list) - 1):
    #     if i % 2 == 0:
    #         new_path_list.append(path_list[i])
    # new_path_list.append(path_list[len(path_list) - 1])

    # path_list = new_path_list

    ordinates_dict = {}


    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        ordinates_dict[pi, pj] = [ext_i, ext_j]

    
    ordinates_dict
    ordinates_list = []
    for key, value in ordinates_dict.items():
        ordinates_list.append(value)
    
    road_lenght = 0
    index = 1
    for _ in ordinates_list[1:]:
        i, j = ordinates_list[index][0], ordinates_list[index][1]
        x, y = ordinates_list[index - 1][0], ordinates_list[index - 1][1]

        if (x - i == 10.0) and (y - j == -10.0):
            road_lenght += math.sqrt(2 * 100)
        else:
            road_lenght += 10 
    
    print(f'Road-length ALGO = [{namming_dict[p]}] = {road_lenght / 1000} kms')
    kilometers.append(road_lenght / 1000)

Road-length ALGO = [A* Chebyshev distance] = 75.97 kms
Road-length ALGO = [A* Diagonal distance] = 69.4 kms
Road-length ALGO = [A* Euclidean distance] = 69.62 kms
Road-length ALGO = [A* Manhattan distance] = 69.17 kms
Road-length ALGO = [A* Proposed] = 76.48 kms
Road-length ALGO = [Dijkstra's algorithm] = 76.45 kms


In [140]:
kilometers

[75.97, 69.4, 69.62, 69.17, 76.48, 76.45]

In [150]:
for k in range(1, 7):

    path = paths_dict[k]
    path_list = []
    with open(f'OUTPUTS/{path}/path.txt', 'r') as f:
        for point in f:
            path_list.append(point)
    
    # print(len(path_list))
    path_list.reverse()

    pixels_dict = {}
    for i in range(len(path_list)):
        point = path_list[i].replace('\n', '').split(' ')
        pi, pj = int(point[0].split(',')[0].split('(')[1]), int(point[1].split(',')[0].split(')')[0])

        ext_i, ext_j = left + (pi * 10), top - (pj * 10)

        # ordinates_dict[(pi, pj)] = extent_matrix[pi][pj]
        # ordinates_dict[pi, pj] = [ext_i, ext_j]
        pixels_dict[ext_i, ext_j] = [pi, pj]
    
    classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland', 8: 'Total'}

    check_pixels = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 0,
        7: 0,
        8: 0,
    }

    df_dict_pixels = {}

    r = shapefile.Reader(f'OUTPUTS/{shape_dict[k]}/shapefiles/test/multipoint.shp')
    feature = r.shapeRecords()[0]
    first = feature.shape.__geo_interface__  

    total_pxs = len(first['coordinates'])
    for cord in first['coordinates']:
        pi, pj = pixels_dict[cord]
        cs = mask_data[pi][pj]

        check_pixels[cs] += 1
        check_pixels[8] += 1
    
    
    for key, value in check_pixels.items():
        df_dict_pixels[classes[key]] = f'{value} pxs'

    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    print(str)

    print(f'ALGORITHM = [{namming_dict[k]}]')
    # print(f'PIXELS LULC {check_pixels}')
    # print(f'PIXELS LULC {df_dict_pixels}')

    for key, value in check_pixels.items():
        check_pixels[key] = np.round(((value / total_pxs) * 100), 2)
    
    # print(f'PERCENTAGE LULC {check_pixels}')

    # classes = {1 : 'Water', 2: 'Dense-forest', 3: 'Sparse-forest', 4: 'Barren-land', 5: 'Urban-land', 6: 'Farmland', 7: 'Fallowland'}
    df_dict_perc = {}
    for key, value in check_pixels.items():
        df_dict_perc[classes[key]] = f'{check_pixels[key]}%'
    
    # print(f'ROAD PERCENTAGE = {df_dict_perc}')

    current_road_distance = kilometers[k - 1]
    road_distance = MAIN_KILO
    # print(f'CURRENT DISTANCE = {road_distance}, OPTIMIZED DISTANCE = {kilometers[k - 1]}')

    df_dict_dist = {}
    recheck = 0
    for key, value in check_pixels.items():
        df_dict_dist[classes[key]] = f'{(np.round(check_pixels[key] * current_road_distance) / 100)} Kms'
        recheck += (check_pixels[key] * road_distance) / 100
    
    # print(f'ROAD KILOMETERS = {df_dict_dist}')
    str = ''
    for i in range(165):
        if i % 2 == 0:
            str += '-'
        else:
            str += '#'
    
    list_of_dicts = [df_dict_pixels, df_dict_perc, df_dict_dist,]
    route_stats = pd.DataFrame(list_of_dicts, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])

    header = ['Water', 'Dense-forest', 'Sparse-forest', 'Barren-land', 'Urban-land', 'Farmland', 'Fallowland', 'Total']
    display(route_stats)

    print(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
    
    print(str)

    with open('df.txt', 'a') as f:

        f.write(str)
        f.write('\n')
        dfAsString = route_stats.to_string(header = header, index = ['PIXELS LULC', 'ROAD PERCENTAGE', 'ROAD KILOMETERS'])
        f.write(dfAsString)
        f.write(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
        # print(f'CURRENT DISTANCE = {np.round(road_distance)} Kms, OPTIMIZED DISTANCE = {kilometers[k - 1]} Kms')
        f.write('\n')
        f.write(str)
            

-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Chebyshev distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,593 pxs,502 pxs,1541 pxs,2075 pxs,263 pxs,179 pxs,2445 pxs,7598 pxs
ROAD PERCENTAGE,7.8%,6.61%,20.28%,27.31%,3.46%,2.36%,32.18%,100.0%
ROAD KILOMETERS,5.93 Kms,5.02 Kms,15.41 Kms,20.75 Kms,2.63 Kms,1.79 Kms,24.45 Kms,75.97 Kms


CURRENT DISTANCE = 94.0 Kms, OPTIMIZED DISTANCE = 75.97 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Diagonal distance]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,790 pxs,185 pxs,1426 pxs,1791 pxs,423 pxs,150 pxs,2176 pxs,6941 pxs
ROAD PERCENTAGE,11.38%,2.67%,20.54%,25.8%,6.09%,2.16%,31.35%,100.0%
ROAD KILOMETERS,7.9 Kms,1.85 Kms,14.25 Kms,17.91 Kms,4.23 Kms,1.5 Kms,21.76 Kms,69.4 Kms


CURRENT DISTANCE = 94.0 Kms, OPTIMIZED DISTANCE = 69.4 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Euclidean distance]


...

CURRENT DISTANCE = 94.0 Kms, OPTIMIZED DISTANCE = 69.17 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [A* Proposed]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,653 pxs,416 pxs,1496 pxs,1939 pxs,356 pxs,182 pxs,2607 pxs,7649 pxs
ROAD PERCENTAGE,8.54%,5.44%,19.56%,25.35%,4.65%,2.38%,34.08%,100.0%
ROAD KILOMETERS,6.53 Kms,4.16 Kms,14.96 Kms,19.39 Kms,3.56 Kms,1.82 Kms,26.06 Kms,76.48 Kms


CURRENT DISTANCE = 94.0 Kms, OPTIMIZED DISTANCE = 76.48 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
ALGORITHM = [Dijkstra's algorithm]


,Water,Dense-forest,Sparse-forest,Barren-land,Urban-land,Farmland,Fallowland,Total
PIXELS LULC,207 pxs,46 pxs,847 pxs,4423 pxs,118 pxs,14 pxs,1991 pxs,7646 pxs
ROAD PERCENTAGE,2.71%,0.6%,11.08%,57.85%,1.54%,0.18%,26.04%,100.0%
ROAD KILOMETERS,2.07 Kms,0.46 Kms,8.47 Kms,44.23 Kms,1.18 Kms,0.14 Kms,19.91 Kms,76.45 Kms


CURRENT DISTANCE = 94.0 Kms, OPTIMIZED DISTANCE = 76.45 Kms
-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-#-
